<a href="https://colab.research.google.com/github/JuanArmas/BigData7RO/blob/main/Practica_Tensor_Flowers_Juan_Armas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad basada en el trabajo del creador de contenido Ringa Tech.

Fuente: https://www.youtube.com/watch?v=JpE4bYyRADI
Modificaciones añadidas:

- Adaptación del script para entrenar el modelo con un dataset de flores, descargado desde https://www.kaggle.com/datasets/imsparsh/flowers-dataset?resource=download.

Por Juan Airam Armas Alemán

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Rutas a las carpetas locales de datos
carpeta_base = '/content/drive/MyDrive/Flowers_Dataset_sin_procesar'
carpeta_entrenamiento = os.path.join(carpeta_base, 'train')
carpeta_prueba = os.path.join(carpeta_base, 'test')

# Subcarpetas para las diferentes flores
flores = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

# Guardar el número de datos de entrenamiento para cada flor
num_flores_entren = {}
num_flores_prueba = {}

for flor in flores:
    carpeta_entren_flores = os.path.join(carpeta_entrenamiento, flor)
    num_entren_flores = len(os.listdir(carpeta_entren_flores))
    num_flores_entren[flor] = num_entren_flores

# Contar todas las imágenes en la carpeta de prueba
num_prueba_total = sum([len(files) for r, d, files in os.walk(carpeta_prueba)])

total_entrenamiento = sum(num_flores_entren.values())
total_prueba = num_prueba_total

In [ ]:
TAMANO_LOTE = 100
TAMANO_IMG = 150

#Aumento de datos (escala, rotacion, blabla)
print("Realizando aumento de datos")
image_gen_entrenamiento = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

#Generacion de datos de entrenamiento FTW
data_gen_entrenamiento = image_gen_entrenamiento.flow_from_directory(batch_size=TAMANO_LOTE,
                                                     directory=carpeta_entrenamiento,
                                                     shuffle=True,
                                                     target_size=(TAMANO_IMG,TAMANO_IMG),
                                                     class_mode='binary')

#Generacion de datos de validacion
image_gen_val = ImageDataGenerator(rescale=1./255)

data_gen_validacion = image_gen_val.flow_from_directory(batch_size=TAMANO_LOTE,
                                                 directory=carpeta_prueba,
                                                 target_size=(TAMANO_IMG, TAMANO_IMG),
                                                 class_mode='binary')

#Modelo!
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

#Compilación
modelo.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#Entrenar la red. Toma un buen rato! Ve por un café ;)
#Oye suscribete al canal!
print("Entrenando modelo...");
epocas=60
history = modelo.fit_generator(
    data_gen_entrenamiento,
    steps_per_epoch=int(np.ceil(total_entrenamiento / float(TAMANO_LOTE))),
    epochs=epocas,
    validation_data=data_gen_validacion,
    validation_steps=int(np.ceil(total_prueba / float(TAMANO_LOTE)))
)

print("Modelo entrenado!");

Realizando aumento de datos
Found 2746 images belonging to 5 classes.
Found 0 images belonging to 0 classes.
Entrenando modelo...


<ipython-input-4-989e8071b852>:60: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = modelo.fit_generator(


Epoch 1/60


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


28/28 [==============================] - 758s 27s/step - loss: 1.4612 - accuracy: 0.3288
Epoch 2/60
28/28 [==============================] - 29s 1s/step - loss: 1.1964 - accuracy: 0.4811
Epoch 3/60
28/28 [==============================] - 31s 1s/step - loss: 1.0494 - accuracy: 0.5666
Epoch 4/60
28/28 [==============================] - 29s 1s/step - loss: 0.9820 - accuracy: 0.6100
Epoch 5/60
28/28 [==============================] - 29s 1s/step - loss: 0.9186 - accuracy: 0.6231
Epoch 6/60
28/28 [==============================] - 29s 1s/step - loss: 0.8920 - accuracy: 0.6442
Epoch 7/60
28/28 [==============================] - 29s 1s/step - loss: 0.8363 - accuracy: 0.6723
Epoch 8/60
28/28 [==============================] - 29s 1s/step - loss: 0.8001 - accuracy: 0.6861
Epoch 9/60
28/28 [==============================] - 29s 1s/step - loss: 0.7839 - accuracy: 0.6977
Epoch 10/60
28/28 [==============================] - 29s 1s/step - loss: 0.7670 - accuracy: 0.6966
Epoch 11/60
28/28 [=========

In [ ]:
#Exportar el modelo en formato h5
modelo.save('Tensor_flowers_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#Para convertirlo a tensorflow.js, primero debemos instalar la libreria
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: ml-dtypes
   

In [ ]:
#Crear carpeta donde se colocaran los archivos resultantes
!mkdir carpeta_salida

In [ ]:
#Realizar la exportacion a la carpeta de salida
!tensorflowjs_converter --input_format keras Tensor_flowers_model.h5 carpeta_salida

2024-05-03 07:37:23.570158: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#Confirmar que en la carpeta de salida se hayan generado los archivos. Deben aparecer archivos "bin" y "json"
!ls carpeta_salida

In [ ]:
#Para descargarlos, da clic del lado izquierdo en el icono de la carpeta
#y expande carpeta_salida. En los archivos utiliza los 3 puntos para descargarlos